In [1]:
import pandas as pd 
import numpy as np 
import nltk 
import itertools
import json

In [2]:
# import ipynb.fs 
# from .full.Functions import clean_tag, cos_sim

In [3]:
df_tags = pd.read_csv("data/tags_selected.csv")

In [4]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(a, b): 
    """Function to calculate the cosine similarity between two vectors. """
    if (norm(a)*norm(b)) == 0: 
        return 0
    return dot(a, b)/(norm(a)*norm(b))

In [5]:
def clean_tag(tag): 
    "Return tokens of the tag."
    tag = tag.replace("-", " ").replace("-", " ").replace("_", " ") #remove "-_"
    tag = "".join(tag.split()) #remove multiple whitespaces 
    
    return tag

In [6]:
df_tags["clean_tag"] = df_tags["tag_name"].apply(clean_tag)
df_tags.head()

,tag_id,book_count,count,tag_name,clean_tag
0,0,7,24,-,
1,1,2,6,--1-,1
2,15,2,6,--6-,6
3,21,2,19,-calif--,calif
4,22,3,27,-d-c--,dc


In [7]:
#df_tags = df_tags.drop(0)
df_tags.head()

,tag_id,book_count,count,tag_name,clean_tag
0,0,7,24,-,
1,1,2,6,--1-,1
2,15,2,6,--6-,6
3,21,2,19,-calif--,calif
4,22,3,27,-d-c--,dc


In [8]:
from flair.embeddings import CharacterEmbeddings, BertEmbeddings, FastTextEmbeddings, WordEmbeddings, Sentence

In [9]:
bert_embeddings = BertEmbeddings("bert-base-uncased")

In [10]:
word_embeddings = WordEmbeddings('glove')

In [11]:
char_embeddings = CharacterEmbeddings()

In [12]:
from flair.embeddings import StackedEmbeddings, BytePairEmbeddings
# init embedding stack
stacked_embeddings = StackedEmbeddings(
    [
#         # standard word embeddings
#         WordEmbeddings('glove'),

#         # character-level features
#         CharacterEmbeddings(),
        bert_embeddings, 
        word_embeddings, 
        char_embeddings
    ]
)

In [13]:
fasttext_embeddings = StackedEmbeddings(
    [
#         # standard word embeddings
#         WordEmbeddings('glove'),

#         # character-level features
#         CharacterEmbeddings(),
#         bert_embeddings, 
        word_embeddings, 
        BytePairEmbeddings("en")
    ]
)

In [14]:
def fasttext_embedding(tag): 
    sentence = Sentence(tag)
    try:
        fasttext_embeddings.embed(sentence)
        embedded = [] 
        for token in sentence:
            embedded.append(token.embedding)
        sum_embedding = sum(embedded).detach().numpy()
        
        return sum_embedding 
    
    except Exception as e: 
        
        return None 
    

In [15]:
def get_embedding(tag): 
    sentence = Sentence(tag)
    try:
        stacked_embeddings.embed(sentence)
        embedded = [] 
        for token in sentence:
            embedded.append(token.embedding)
        sum_embedding = sum(embedded).detach().numpy()
        
        return sum_embedding 
    
    except Exception as e: 
        
        return None    

In [16]:
df_tags["stacked_embed"] = df_tags["clean_tag"].apply(get_embedding)

2020-02-12 16:46:45,236 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-02-12 16:47:26,333 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?


In [17]:
df_tags.head()

,tag_id,book_count,count,tag_name,clean_tag,stacked_embed
0,0,7,24,-,,None
1,1,2,6,--1-,1,"[-0.32219, 0.64632, 0.027312, 0.36579, -0.0026..."
2,15,2,6,--6-,6,"[-0.10186, 0.38768, 0.077677, 0.26919, -0.1496..."
3,21,2,19,-calif--,calif,"[-0.51205, 0.1907, 0.57485, -0.92168, -0.14986..."
4,22,3,27,-d-c--,dc,"[0.021061, -0.11286, 0.36684, -0.45036, 0.9971..."


In [18]:
df_tags.dropna(inplace=True)

In [19]:
#stacked embeddings 
count = 0
links_stacked_embeddings = [] 
for pair in itertools.combinations(list(df_tags.index), 2):
    similarity = cos_sim(df_tags.stacked_embed[pair[0]], 
                      df_tags.stacked_embed[pair[1]])
    if similarity >=0.90: 
#         print("ids: ", pair)
        print("tags: ", df_tags.tag_name[pair[0]], df_tags.tag_name[pair[1]], "cosine similarity: ", similarity)
#         print("cosine similarity: ", similarity)
        count +=1  
    if count >=100: 
        break

tags:  --1- 1 cosine similarity:  0.99999994
tags:  --6- 6 cosine similarity:  1.0000001
tags:  -calif-- calif cosine similarity:  1.0
tags:  -d-c-- d-c cosine similarity:  1.0000001
tags:  -d-c-- dc cosine similarity:  1.0000001
tags:  -dean dean cosine similarity:  0.9999999
tags:  -fictitious fictitious cosine similarity:  0.9999999
tags:  -gr- gr cosine similarity:  1.0
tags:  -mass-- mass cosine similarity:  1.0
tags:  -n-y-- n-y cosine similarity:  0.9999999
tags:  -post post cosine similarity:  0.9999999
tags:  04-babies-and-toddlers 04-caldecott cosine similarity:  0.90305835
tags:  04-babies-and-toddlers 04-preschoolers cosine similarity:  0.92215663
tags:  04-caldecott 04-preschoolers cosine similarity:  0.91588646
tags:  04-caldecott 04-realistic-fiction cosine similarity:  0.91104376
tags:  04-preschoolers 04-realistic-fiction cosine similarity:  0.92697734
tags:  1-000-books-before-kindergarten 1000-books-before-kindergarten cosine similarity:  0.99999994
tags:  1-000-book

tags:  1001-book-list 1001-books-2006 cosine similarity:  0.90550447
tags:  1001-book-list 1001-books-list cosine similarity:  0.97380775
tags:  1001-book-list 1001-books-read cosine similarity:  0.92567813
tags:  1001-book-list 1001-kids-books cosine similarity:  0.90357643
tags:  1001-book-list 1001-list cosine similarity:  0.9267062
tags:  1001-book-list 1001-read cosine similarity:  0.91601646
tags:  1001-books 1001-books-2006 cosine similarity:  0.926364
tags:  1001-books 1001-books-to-read cosine similarity:  0.9203249
tags:  1001-books 1001-books-you-must-read cosine similarity:  0.9179701
tags:  1001-books 1001-children-s-books cosine similarity:  0.9065041
tags:  1001-books 1001-childrens-books cosine similarity:  0.9065041
tags:  1001-books 1001-libri-da-leggere cosine similarity:  0.90699404
tags:  1001-books 1001-read cosine similarity:  0.90316445
tags:  1001-books 1001-to-read cosine similarity:  0.9095574
tags:  1001-books 1001books cosine similarity:  1.0
tags:  1001-bo

tags:  1001-children-s-books 1001-children-s-books-you-must-read cosine similarity:  0.9524248
tags:  1001-children-s-books 1001-childrens-books cosine similarity:  1.0
tags:  1001-children-s-books 1001-libri-da-leggere cosine similarity:  0.91930026
tags:  1001-children-s-books 1001-read cosine similarity:  0.9059227
tags:  1001-children-s-books 1001books cosine similarity:  0.9065041
tags:  1001-children-s-books-you-must-read 1001-childrens-books cosine similarity:  0.9524248
tags:  1001-children-s-books-you-must-read 1001-import cosine similarity:  0.9105793
tags:  1001-children-s-books-you-must-read 1001-kids-5plus cosine similarity:  0.90700907
tags:  1001-children-s-books-you-must-read 1001-libri-da-leggere cosine similarity:  0.94551986
tags:  1001-children-s-books-you-must-read 1001-read cosine similarity:  0.90662724
tags:  1001-children-s-books-you-must-read 1001-to-read cosine similarity:  0.9227363
tags:  1001-children-s-books-you-must-read 1001-to-read-before-you-die cosin

tags:  2012-and-before 2013-and-before cosine similarity:  0.91772366
tags:  2012-book-challenge 2012-book-list cosine similarity:  0.9342537
tags:  2012-book-challenge 2012-books-read cosine similarity:  0.94261605
tags:  2012-book-challenge 2012-challenge cosine similarity:  0.9427476
tags:  2012-book-challenge 2012-debut-author-challenge cosine similarity:  0.9190369
tags:  2012-book-challenge 2012-forty-book-challenge cosine similarity:  0.9648542
tags:  2012-book-challenge 2012-reading-challenge cosine similarity:  0.9389817
tags:  2012-book-challenge 2012-reading-list cosine similarity:  0.9070526
tags:  2012-book-challenge 2012-release cosine similarity:  0.90819216
tags:  2012-book-challenge 2012-releases cosine similarity:  0.9054779
tags:  2012-book-challenge 2013-book-challenge cosine similarity:  0.9198408
tags:  2012-book-list 2012-books-read cosine similarity:  0.90607095
tags:  2012-book-list 2012-forty-book-challenge cosine similarity:  0.91089106
tags:  2012-book-list 

tags:  2014-challenge 2014-movies cosine similarity:  0.9069014
tags:  2014-challenge 2014-publication cosine similarity:  0.9310593
tags:  2014-challenge 2014-reading-challenge cosine similarity:  0.9472613
tags:  2014-challenge 2014-release cosine similarity:  0.915127
tags:  2014-challenge 2014-releases cosine similarity:  0.9167749
tags:  2014-challenge 2014-tournament-of-books cosine similarity:  0.9241041
tags:  2014-favorites 2015-favorites cosine similarity:  0.91669905
tags:  2014-movies 2014-publication cosine similarity:  0.92864054
tags:  2014-movies 2014-release cosine similarity:  0.927559
tags:  2014-movies 2014-releases cosine similarity:  0.93484056
tags:  2014-movies 2014-tournament-of-books cosine similarity:  0.9196551
tags:  2014-publication 2014-reading-challenge cosine similarity:  0.9164338
tags:  2014-publication 2014-release cosine similarity:  0.92107856
tags:  2014-publication 2014-releases cosine similarity:  0.922735
tags:  2014-publication 2014-tournament

In [ ]:
#probably try weighted shceme connected to similarity score, for example weight = similarity * MAX_WEIGHT 
MAX_WEIGHT = 1000000
links_stacked_embeddings = [] 
for pair in itertools.combinations(list(df_tags.index), 2):
    similarity = cos_sim(df_tags.stacked_embed[pair[0]], 
                      df_tags.stacked_embed[pair[1]])
    if similarity >=0.90: 
        links_stacked_embeddings.append((df_tags.tag_name[pair[0]], df_tags.tag_name[pair[1]], similarity*1000000))

#         print("ids: ", pair)
#         print("tags: ", df_tags.tag_name[pair[0]], df_tags.tag_name[pair[1]], "cosine similarity: ", similarity)
#         print("cosine similarity: ", similarity)

In [ ]:
links_stacked_embeddings

In [ ]:
def save_edges(edgelist, outfile, weights=False):
    """Write the edges in a file in a suitable for Node2vec form. """
    with open(outfile, "w+") as f:
        for edge in edgelist:
            f.write(str(edge[0]))
            f.write(" ")
            f.write(str(edge[1]))
            if weights:
                    f.write(" ")
                    f.write(str(edge[2]))
            f.write("\n")    
save_edges(concat_edges, "edges/tag_to_tag_w_stacked.edgelist", weights=True)

In [33]:
#fasttext embeddings - they also have problems with numbers 
count = 0
for pair in itertools.combinations(list(df_tags.index), 2):
    similarity = cos_sim(df_tags.fasttext_embed[pair[0]], 
                      df_tags.fasttext_embed[pair[1]])
    if similarity >=0.90: 
#         print("ids: ", pair)
        print("tags: ", df_tags.tag_name[pair[0]], df_tags.tag_name[pair[1]], "cosine similarity: ", similarity)
#         print("cosine similarity: ", similarity)
        count +=1  
    if count >=200: 
        break

tags:  --1- --6- cosine similarity:  0.947755
tags:  --1- 1 cosine similarity:  1.0000001
tags:  --1- 2 cosine similarity:  0.98450464
tags:  --1- 3 cosine similarity:  0.9758283
tags:  --1- 4 cosine similarity:  0.96548015
tags:  --1- 5 cosine similarity:  0.9543705
tags:  --1- 6 cosine similarity:  0.947755
tags:  --1- 7 cosine similarity:  0.93380165
tags:  --1- 9 cosine similarity:  0.9198399
tags:  --6- 1 cosine similarity:  0.947755
tags:  --6- 10 cosine similarity:  0.91410637
tags:  --6- 11 cosine similarity:  0.9069754
tags:  --6- 12 cosine similarity:  0.91766423
tags:  --6- 13 cosine similarity:  0.91304755
tags:  --6- 2 cosine similarity:  0.9679983
tags:  --6- 3 cosine similarity:  0.9850942
tags:  --6- 4 cosine similarity:  0.99386436
tags:  --6- 5 cosine similarity:  0.99220127
tags:  --6- 6 cosine similarity:  1.0
tags:  --6- 7 cosine similarity:  0.9905028
tags:  --6- 9 cosine similarity:  0.9838043
tags:  -calif-- calif cosine similarity:  1.0000001
tags:  -d-c-- d-c 

tags:  100-books 100-must-read-books cosine similarity:  0.9999999
tags:  100-books 311-books cosine similarity:  0.9999999
tags:  100-books 501-must-read-books cosine similarity:  0.9999999
tags:  100-books 52-books cosine similarity:  0.92242444
tags:  100-books-every-man-should-read 100-novels-everyone-should-read cosine similarity:  1.0000001
tags:  100-books-to-read-before-you-die 50-books-to-read-before-you-die cosine similarity:  0.933416
tags:  100-essential-novels 100-greatest-novels cosine similarity:  1.0
tags:  100-essential-novels 50-essential-cult-novels cosine similarity:  0.9131896
tags:  100-greatest-novels 50-essential-cult-novels cosine similarity:  0.9131896
tags:  100-must-read-books 311-books cosine similarity:  0.9999999
tags:  100-must-read-books 501-must-read-books cosine similarity:  0.9999999
tags:  100-must-read-books 52-books cosine similarity:  0.92242444
tags:  1000-books 1001-books cosine similarity:  1.0000001
tags:  1000-books 1001-children-s-books cos

In [85]:
df_tags.to_csv("tags_stacked_embeddings.csv")

In [88]:
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


# Compute DBSCAN
def evaluate_dbscan(X, eps=0.5, min_samples=10, metric='cosine'): 
    db = DBSCAN(eps=eps, min_samples=min_samples, metric=metric).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
#     print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
#     print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
#     print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
#     print("Adjusted Rand Index: %0.3f"
#           % metrics.adjusted_rand_score(labels_true, labels))
#     print("Adjusted Mutual Information: %0.3f"
#           % metrics.adjusted_mutual_info_score(labels_true, labels))
    print("Silhouette Coefficient: %0.3f"
          % metrics.silhouette_score(X, labels))
    
    return metrics.silhouette_score(X, labels), labels

In [91]:
df_tags_1000 = df_tags[0:1000]

In [92]:
X = list(df_tags_1000.stacked_embed.values)

In [101]:
evaluate_dbscan(X, eps=0.25, min_samples=2, metric='cosine')[0]

Estimated number of clusters: 74
Estimated number of noise points: 175
Silhouette Coefficient: 0.105


0.10487504

In [103]:
df_tags_1000["db_scan_labels"] = pd.Series(evaluate_dbscan(X, eps=0.25, min_samples=2, metric='cosine')[1])

Estimated number of clusters: 74
Estimated number of noise points: 175
Silhouette Coefficient: 0.105


C:\Users\melania\Anaconda3\envs\rec-sys-test\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [105]:
for i in range(-1, 74): 
    print("cluster", i, ":", list(df_tags_1000[df_tags_1000["db_scan_labels"]==i]["tag_name"].values))   

cluster -1 : ['--6-', '-calif--', '-gr-', '-mass--', '-n-y--', '-post', '0-owned', '00-in-class', '1001books', '13', '14th-century', '1500s', '1700s', '19th', '19th-century-literature', '2017-tournament-of-books', '4-6', '4000-books', '50-states', '501', '52-books', '53-books-you-won-t-be-able-to-put-d', '5360-summer-12', '60s', '9-11', '_owned', 'a', 'a-course-in-miracles', 'a-galaxy-far-far-away', 'a-game-of-thrones', 'a-j-jacobs', 'a-kindle', 'a-little-golden-book', 'a-meredith-walters', 'a-must-read', 'a-n-roquelaure', 'a-own', 'a-shade-of-vampire', 'a-song-of-fire-and-ice', 'a-song-of-ice-and-fire', 'a-w-tozer', 'aa', 'aaa', 'ab-better-cover-1-maybe', 'abandon', 'abandonados', 'abandoned', 'abandoned-children', 'abarat', 'abcs', 'abduction', 'abercrombie', 'abercrombie-joe', 'abhorsen', 'ability', 'aboriginal', 'about-writing', 'abraham-hicks', 'abraham-lincoln', 'abridged', 'abuse', 'abused-heroine', 'acceptance', 'ace', 'aching', 'activity', 'actor-actress', 'adam-dalgliesh-seri

# Genres to embeddings 

In [17]:
#Open genre instanses file and extract all genres
with open("onto_files/all_genres.json") as f:
    data = json.load(f)
genres = dict()
for res in data['results']['bindings']: 
    genres[res['label']['value']] =  res['genre']['value']

In [18]:
genres_embed = dict() 
for gen in genres.keys():
    genres_embed[gen] = get_embedding(gen)

In [19]:
genres_embed

{'realist': array([-0.034907  ,  0.71737   ,  0.05484   , ...,  0.04654844,
        -0.08071265, -0.3364035 ], dtype=float32),
 'fairytale': array([ 0.31262   , -0.11606   ,  0.44541   , ..., -0.19821635,
        -0.13506939,  0.05805377], dtype=float32),
 'guidebook': array([-0.73817   ,  0.065518  ,  0.70767   , ...,  0.24377415,
         0.00297781, -0.13245656], dtype=float32),
 'treatise': array([-0.26633   ,  0.49781   , -0.25101   , ..., -0.19821635,
        -0.13506937,  0.05805375], dtype=float32),
 'novella': array([-0.29875   , -0.15561   , -0.089751  , ..., -0.15571286,
        -0.1009185 , -0.08697555], dtype=float32),
 'polemic': array([-0.16032   ,  0.55275   , -0.39763   , ...,  0.04360614,
        -0.05267817,  0.01706021], dtype=float32),
 'magic realist': array([ 0.447693  ,  0.50444996,  0.63452995, ...,  0.09015459,
        -0.13339081, -0.31934333], dtype=float32),
 'panegyric': array([-0.34645   ,  0.2992    , -0.20547   , ...,  0.04360615,
        -0.05267817,  

In [114]:
list(genres.keys())

['realist',
 'fairytale',
 'guidebook',
 'treatise',
 'novella',
 'polemic',
 'magic realist',
 'panegyric',
 'slave narrative',
 'dramatic monologue',
 'popular',
 'genealogy',
 'bergamasque',
 'dissertation',
 'epitaph',
 'hagiography',
 'myth',
 'school fiction',
 'conversion narrative',
 'drama',
 'sentimental',
 'bouts-rimÃ©s',
 'sequel',
 'dream vision',
 'philosophy',
 'answer',
 'oral history',
 'allegory',
 'fiction',
 'letters from the dead to the living',
 'military',
 'feminist theory',
 'science fiction',
 'industrial novel',
 'translation',
 'theatre of cruelty',
 'eclogue',
 'verse novel',
 'temperance',
 'pastoral',
 'haiku',
 'tragedy',
 'sermon',
 'bestiary',
 'comedy of intrigue',
 'oratorio',
 'theatre of the absurd',
 'young adult writing',
 'performance poetry',
 'psalm',
 'immigration',
 'dialogue or debate',
 'domestic',
 'elegy',
 'gardening',
 'memoir',
 'monologue',
 'black comedy',
 'encyclopaedia',
 'lais',
 'harlequinade',
 'closet drama',
 'parable',
 'ep

In [20]:
d = {'genre_name': list(genres.keys()), 'genre_node': list(genres.values()), 'stacked_embed': list(genres_embed.values())}
df_genres = pd.DataFrame(data=d)
df_genres

,genre_name,genre_node,stacked_embed
0,realist,http://sparql.cwrc.ca/ontologies/genre#realist,"[-0.034907, 0.71737, 0.05484, 0.31859, -0.2407..."
1,fairytale,http://sparql.cwrc.ca/ontologies/genre#fairytale,"[0.31262, -0.11606, 0.44541, 0.15918, -0.10457..."
2,guidebook,http://sparql.cwrc.ca/ontologies/genre#guidebook,"[-0.73817, 0.065518, 0.70767, -0.0059764, 0.08..."
3,treatise,http://sparql.cwrc.ca/ontologies/genre#treatise,"[-0.26633, 0.49781, -0.25101, 1.1087, 0.30035,..."
4,novella,http://sparql.cwrc.ca/ontologies/genre#novella,"[-0.29875, -0.15561, -0.089751, 0.29644, 0.593..."
...,...,...,...
279,social reform,http://sparql.cwrc.ca/ontologies/genre#socialR...,"[-0.40094, 0.84253997, -0.72290003, -0.19873, ..."
280,erotica,http://sparql.cwrc.ca/ontologies/genre#erotica,"[0.47574, 0.22808, 0.10798, 0.12213, 0.19596, ..."
281,environmental,http://sparql.cwrc.ca/ontologies/genre#environ...,"[-0.16813, 0.18316, 0.00080039, -0.43273, 0.54..."
282,commentary,http://sparql.cwrc.ca/ontologies/genre#commentary,"[-0.86444, 0.34262, 0.27998, -1.0168, 0.30103,..."


In [39]:
df_tags.dropna(inplace=True)
df_tags.head()

,tag_id,book_count,count,tag_name,clean_tag,stacked_embed
1,1,2,6,--1-,1,"[-0.32219, 0.64632, 0.027312, 0.36579, -0.0026..."
2,15,2,6,--6-,6,"[-0.10186, 0.38768, 0.077677, 0.26919, -0.1496..."
3,21,2,19,-calif--,calif,"[-0.51205, 0.1907, 0.57485, -0.92168, -0.14986..."
4,22,3,27,-d-c--,dc,"[0.021061, -0.11286, 0.36684, -0.45036, 0.9971..."
5,23,8,50,-dean,dean,"[0.23422, -0.23815, -0.70271, -0.79144, 0.3894..."


In [40]:
count = 0
for pair in itertools.product(list(df_tags.index), list(df_genres.index)):
    similarity = cos_sim(df_tags.stacked_embed[pair[0]], 
                      df_genres.stacked_embed[pair[1]])
    if similarity >=0.85 and df_genres.genre_name[pair[1]] not in ["paranormal", "lullaby"]: 
#         print("ids: ", pair)
        print("tags: ", df_tags.tag_name[pair[0]], df_genres.genre_name[pair[1]], "cosine similarity: ", similarity)
#         print("cosine similarity: ", similarity)
        count +=1  
    if count >=100: 
        break

tags:  abduction liturgy cosine similarity:  0.85352737
tags:  abduction imitation cosine similarity:  0.87257785
tags:  abridged abridgement cosine similarity:  0.8893644
tags:  accidents imitation cosine similarity:  0.85030174
tags:  adaptation adaptation cosine similarity:  0.99926746
tags:  adultery liturgy cosine similarity:  0.8716073
tags:  aging imitation cosine similarity:  0.86352664
tags:  airplane announcement cosine similarity:  0.8665184
tags:  allegories allegory cosine similarity:  0.9289074
tags:  allegory allegory cosine similarity:  0.9993681
tags:  anarchy utopia cosine similarity:  0.85171586
tags:  apocalyptic announcement cosine similarity:  0.85378814
tags:  army military cosine similarity:  0.8575541
tags:  auto-biography memoir cosine similarity:  0.87609637
tags:  auto-biography autobiography cosine similarity:  0.99937886
tags:  autobiography memoir cosine similarity:  0.87609637
tags:  autobiography autobiography cosine similarity:  0.99937886
tags:  bible

- "lullaby" and "thanksgiving" give a lot of false positives 

## Analysis: 
#### wrong: 
   - autism paranormal cosine similarity:  0.90457016 
   - behavioral paranormal cosine similarity:  0.90563065
   -  futuristic paranormal cosine similarity:  0.9039197
  -   psychedelic paranormal cosine similarity:  0.90347755
#### catched meaning: 
    - tale lullaby cosine similarity:  0.90073854
    -  wonderland lullaby cosine similarity:  0.91632396

In [41]:
def find_genre(embedding, threshold=90): 
    for index in list(df_genres.index):
        similarity = cos_sim(embedding, 
                      df_genres.stacked_embed[index])
        if similarity >threshold: 
            return df_genres.genre_name[index]
        
    return None 

In [42]:
df_tags["genre_name"] = df_tags["stacked_embed"].apply(find_genre)

In [43]:
#define some manual tests: list(product(test_tup, test_list))
# ????m
df_tags.dropna(inplace=True)
df_tags

,tag_id,book_count,count,tag_name,clean_tag,stacked_embed,genre_name


In [171]:
df_tags.to_csv("data/tags_stacked_embeddings_genres.csv")

In [47]:
#df_tags = pd.read_csv("data/tags_stacked_embeddings_genres.csv")
df_tags.dropna(inplace = True)
df_tags

,Unnamed: 0,tag_id,book_count,count,tag_name,clean_tag,stacked_embed,genre_name
573,575,1584,17,62,adaptation,adaptation,[ 0.02827154 -0.39966932 -0.4262851 ... 0.13...,adaptation
797,799,2112,2,20,allegories,allegories,[-0.7937539 -0.0359518 -0.28845835 ... 0.14...,allegory
798,800,2113,33,1577,allegory,allegory,[-7.8196716e-01 2.9064909e-01 -2.5628287e-01 ...,allegory
1460,1462,3586,25,2005,autism,autism,[-0.51119936 -0.59016585 0.05876559 ... -0.12...,paranormal
1467,1469,3596,265,3130,auto-biography,autobiography,[-0.02404164 -0.08033854 -0.32767 ... 0.20...,autobiography
...,...,...,...,...,...,...,...,...
14932,14934,31667,11,268,utopia,utopia,[-0.3878106 -0.21069324 -0.4752291 ... 0.23...,utopia
15155,15157,32130,1044,66056,war,war,[ 0.5518989 0.50877196 -0.49524078 ... 0.10...,war
15156,15158,32131,2,5,war-,war,[ 0.5518989 0.50877196 -0.49524078 ... 0.10...,war
15276,15278,32353,115,9899,western,western,[-0.83105254 -0.10517035 -0.5715087 ... 0.13...,western


In [50]:
pd.set_option('display.max_rows', 1000)
display(df_tags)

,Unnamed: 0,tag_id,book_count,count,tag_name,clean_tag,stacked_embed,genre_name
573,575,1584,17,62,adaptation,adaptation,[ 0.02827154 -0.39966932 -0.4262851 ... 0.13...,adaptation
797,799,2112,2,20,allegories,allegories,[-0.7937539 -0.0359518 -0.28845835 ... 0.14...,allegory
798,800,2113,33,1577,allegory,allegory,[-7.8196716e-01 2.9064909e-01 -2.5628287e-01 ...,allegory
1460,1462,3586,25,2005,autism,autism,[-0.51119936 -0.59016585 0.05876559 ... -0.12...,paranormal
1467,1469,3596,265,3130,auto-biography,autobiography,[-0.02404164 -0.08033854 -0.32767 ... 0.20...,autobiography
1477,1479,3611,655,36375,autobiography,autobiography,[-0.02404164 -0.08033854 -0.32767 ... 0.20...,autobiography
1751,1753,4199,10,161,behavioral,behavioral,[-0.44232374 -0.56229657 -0.0422841 ... -0.01...,paranormal
1885,1887,4514,65,1497,bildungsroman,bildungsroman,[ 0.05046185 -1.1567993 0.93424946 ... 0.13...,bildungsroman
1886,1888,4516,8,99,bildungsromans,bildungsromans,[ 0.15128914 -0.8636121 0.56865 ... 0.14...,bildungsroman
1933,1935,4605,1109,172444,biography,biography,[ 0.564145 0.16239157 -0.29985443 ... 0.20...,biography


In [58]:
#Drop mistakes
#df_tags.drop(1751, inplace=True)
df_tags.drop(3648, inplace=True)
#df_tags.drop(5779, inplace=True)
#df_tags.drop(15451, inplace=True)

In [59]:
df_tags

,Unnamed: 0,tag_id,book_count,count,tag_name,clean_tag,stacked_embed,genre_name
573,575,1584,17,62,adaptation,adaptation,[ 0.02827154 -0.39966932 -0.4262851 ... 0.13...,adaptation
797,799,2112,2,20,allegories,allegories,[-0.7937539 -0.0359518 -0.28845835 ... 0.14...,allegory
798,800,2113,33,1577,allegory,allegory,[-7.8196716e-01 2.9064909e-01 -2.5628287e-01 ...,allegory
1460,1462,3586,25,2005,autism,autism,[-0.51119936 -0.59016585 0.05876559 ... -0.12...,paranormal
1467,1469,3596,265,3130,auto-biography,autobiography,[-0.02404164 -0.08033854 -0.32767 ... 0.20...,autobiography
1477,1479,3611,655,36375,autobiography,autobiography,[-0.02404164 -0.08033854 -0.32767 ... 0.20...,autobiography
1885,1887,4514,65,1497,bildungsroman,bildungsroman,[ 0.05046185 -1.1567993 0.93424946 ... 0.13...,bildungsroman
1886,1888,4516,8,99,bildungsromans,bildungsromans,[ 0.15128914 -0.8636121 0.56865 ... 0.14...,bildungsroman
1933,1935,4605,1109,172444,biography,biography,[ 0.564145 0.16239157 -0.29985443 ... 0.20...,biography
2529,2531,5951,377,73249,business,business,[ 0.31106147 -0.48132646 0.09144838 ... 0.14...,business


In [39]:
#Try on the sample
df_sample= pd.read_csv('test_data/tags_sample.csv')
df_sample["clean_tag"] = df_sample["tag_name"].apply(clean_tag)
df_sample["stacked_embed"] = df_sample["clean_tag"].apply(get_embedding)
df_sample.head()

,Unnamed: 0,tag_id,book_count,count,tag_name,clean_tag,stacked_embed
0,13450,28446,2,20,sports-romances,sportsromances,"[0.062709995, -0.23355573, -0.0015637614, -0.1..."
1,13451,28448,2,19,sports-star,sportsstar,"[-0.18590917, -0.14466536, -0.51539165, 0.6037..."
2,13452,28450,3,30,sports-theme,sportstheme,"[0.089826696, 0.023328137, -0.0051116906, -0.1..."
3,13453,28454,4,41,sporty,sporty,"[0.26056933, 0.06663957, 0.4998312, -0.0864978..."
4,13454,28456,3,24,spouses,spouses,"[0.72043246, -0.06357735, 0.3398019, 0.1811603..."


In [60]:
df_tags.to_csv("data/tags_stacked_embeddings_genres.csv")

In [40]:
import pickle
file_sim_set = "test_data/simipar_tags_test_set.pickle"
file_all_set = "test_data/all_tags_test_set.pickle"

with open(file_sim_set, mode='rb') as f_sim:
    similar_set = pickle.load(f_sim)
    
with open(file_all_set, 'rb') as f_all:
    all_pairs_set = pickle.load(f_all)

In [41]:
not_similar_set =  all_pairs_set - similar_set
print("Similar pairs: ", len(similar_set))
print("All pairs: ", len(all_pairs_set))
print("Not similar pairs: ", len(not_similar_set))
print("percent in relation: ", len(similar_set) / len(all_pairs_set))

Similar pairs:  64
All pairs:  4950
Not similar pairs:  4886
percent in relation:  0.01292929292929293


In [42]:
def evaluate(similar_pred, not_similar_pred, similar_set, not_similar_set):
    print("predicted similar: ", len(similar_pred))
    print("predicted not similar: ", len(not_similar_pred))
    
    TP_set = similar_pred & similar_set
    TP = len(TP_set)
    print("True positives: ", TP)
    
    FP_set = similar_pred - similar_set
    FP = len(FP_set)
    print("False positives: ", FP)
    
    TN_set = not_similar_pred & not_similar_set
    TN = len(TN_set)
    print("True negative: ", TN)
    
    FN_set = not_similar_pred - not_similar_set
    FN = len(FN_set)
    print("False negatives: ", FN)
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = (2 * precision * recall) / (precision + recall)  
    
    
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1-score: ", F1)
                            
    ALL_P = len(similar_pred)
    ALL_N = len(not_similar_pred)
    assert(TP + FP == ALL_P)
    assert(TN + FN == ALL_N)
    
    return TP_set, FP_set, TN_set, FN_set

In [43]:
def predict(df_sample, col_name, similarity_func, threshold):
    similar_pred = set()
    not_similar_pred = set()

    for pair in itertools.combinations(list(df_sample.index), 2):
        similarity = similarity_func(df_sample[col_name][pair[0]], 
                          df_sample[col_name][pair[1]])

        if similarity >threshold: 
            similar_pred.add((df_sample.tag_id[pair[0]], df_sample.tag_id[pair[1]]))

        else: 
            not_similar_pred.add((df_sample.tag_id[pair[0]], df_sample.tag_id[pair[1]]))
            
    return similar_pred, not_similar_pred 

In [53]:
similar_pred, not_similar_pred = predict(df_sample, col_name="stacked_embed", similarity_func=cos_sim, threshold=0.90)
TP_set, FP_set, TN_set, FN_set = evaluate(similar_pred, not_similar_pred, similar_set, not_similar_set)

predicted similar:  95
predicted not similar:  4855
True positives:  14
False positives:  81
True negative:  4805
False negatives:  50
Precision:  0.14736842105263157
Recall:  0.21875
F1-score:  0.17610062893081757


In [45]:
df_tag_index = df_sample.set_index("tag_id", inplace=False)
df_tag_index.head()

,Unnamed: 0,book_count,count,tag_name,clean_tag,stacked_embed
tag_id,,,,,,
28446,13450,2,20,sports-romances,sportsromances,"[0.062709995, -0.23355573, -0.0015637614, -0.1..."
28448,13451,2,19,sports-star,sportsstar,"[-0.18590917, -0.14466536, -0.51539165, 0.6037..."
28450,13452,3,30,sports-theme,sportstheme,"[0.089826696, 0.023328137, -0.0051116906, -0.1..."
28454,13453,4,41,sporty,sporty,"[0.26056933, 0.06663957, 0.4998312, -0.0864978..."
28456,13454,3,24,spouses,spouses,"[0.72043246, -0.06357735, 0.3398019, 0.1811603..."


In [48]:
def display(df_tag_index, pair_set, vector_col_name="stacked_embed"):
    """Display the tag names in a set of tag pairs. """
    for pair in pair_set: 
        cossim = cos_sim(df_tag_index.loc[pair[0]][vector_col_name], df_tag_index.loc[pair[1]][vector_col_name])
        print(df_tag_index.loc[pair[0]]["tag_name"], df_tag_index.loc[pair[1]]["tag_name"], cossim)

In [49]:
print("True positives: ")
display(df_tag_index, TP_set)

True positives: 
steam-punk steampunk 1.0
star-wars-cannon star-wars-canon 0.950691
steel--danielle steel-danielle 1.0000001
star-wars star-wars-books 0.9151684
start-up startup 1.0000001
star-wars-own star-wars-owned 0.9411336
spy-novel spy-novels 0.92447823
star-wars starwars 1.0000001
start-ups startups 1.0
star_wars starwars 1.0000001
star-wars star_wars 1.0000001
started-but-didn-t-finish started-didn-t-finish 0.92225826
steinbeck steinbeck-john 0.9214851
started-but-didn-t-finish started-but-not-finished 0.90129113


In [54]:
print("False positives: ")
display(df_tag_index, FP_set)

False positives: 
star-wars-canon star-wars-universe 0.9143688
star-wars-old-republic-era star-wars-read 0.9247502
star-wars-canon star-wars-disney-canon 0.9314141
star-wars-new-canon star-wars-read 0.9141731
star-wars-cannon star-wars-to-read 0.9266605
star-wars-own star-wars-to-read 0.9149849
star-wars-collection star-wars-legends 0.9430578
star-wars-cannon star-wars-old-republic-era 0.93227005
star-wars-books star_wars 0.9151684
star-wars-canon star-wars-read 0.92639154
star-wars-read star-wars-to-read 0.9608281
star-wars-canon star-wars-own 0.90475345
star-wars-new-canon star-wars-own 0.90853137
star-wars-legends star-wars-legends-novels 0.95364034
star-wars-new-canon star-wars-universe 0.90374494
star-wars-collection star-wars-novels 0.92016405
star-wars-cannon star-wars-new-canon 0.90754044
star-wars-cannon star-wars-disney-canon 0.9435475
star-wars-disney-canon star-wars-read 0.94339883
star-wars-disney-canon star-wars-old-republic-era 0.9344275
star-wars-canon star-wars-to-read

In [57]:
print(FP_set | TP_set)

{(28558, 28561), (28545, 28547), (28554, 28561), (28544, 28564), (28559, 28564), (28546, 28552), (28482, 28484), (28541, 28542), (28542, 28567), (28545, 28561), (28561, 28564), (28554, 28559), (28544, 28554), (28545, 28564), (28560, 28561), (28556, 28564), (28547, 28554), (28545, 28558), (28552, 28554), (28544, 28561), (28556, 28561), (28544, 28548), (28547, 28560), (28552, 28560), (28552, 28559), (28553, 28558), (28580, 28594), (28446, 28450), (28552, 28565), (28546, 28564), (28545, 28548), (28544, 28556), (28541, 28597), (28547, 28553), (28546, 28547), (28545, 28553), (28567, 28597), (28546, 28558), (28546, 28561), (28544, 28545), (28547, 28556), (28558, 28560), (28545, 28556), (28549, 28560), (28552, 28556), (28477, 28479), (28554, 28558), (28541, 28567), (28583, 28586), (28544, 28553), (28547, 28548), (28546, 28548), (28553, 28556), (28615, 28616), (28559, 28560), (28545, 28565), (28623, 28624), (28544, 28558), (28545, 28559), (28552, 28553), (28554, 28565), (28546, 28556), (28544,

In [58]:
pos_set = FP_set | TP_set
len(pos_set)

95

In [ ]:
def remove_redundant_links(edges): 
    """For every two edges (a, b), (b, c), remove edge (a, c), if it is in the set."""
    for pair_of_edges in iterpools.product(list(edges), list(edges)): 
        if pair_of_edges[0][1] == 